# Equity Risk Factor Modelling

# Setup

Assuming some input parameters

In [11]:
import numpy as np 
import polars as pl 
import matplotlib.pyplot as plt 


# Model parameters
S0: float = 100.0 # initial spot 
r: float = 0.02   # risk-free rate 
q: float = 0.01   # dividend yield 
T: float = 1.0    # horizon (years) 
n_paths: int = 10_000 # number of Monte Carlo paths 
n_steps: int = 252    # Number of steps in one year
time_grid = np.linspace(start=0.0, stop=T, num=n_steps + 1) # Number of interval is n_steps but number of points is n_steps + 1 

# Setting Seed
rng= np.random.default_rng(seed=42) 

# Polars DataFrame to hold simulated results 
results_schema: dict[str,any] = { "path_id": pl.Int32, "time": pl.Float64, "S": pl.Float64 } 
simulated_paths = pl.DataFrame(schema=results_schema) 

# Plot style
plt.style.use("seaborn-v0_8-whitegrid") 
print("Setup complete. Polars and parameters initialized.")

Setup complete. Polars and parameters initialized.


# Basic Model 
This will be used by Heston, SABR or Local Volatility

In [12]:
from abc import ABC, abstractmethod
import polars as pl
import numpy as np

class BaseModel(ABC):
    """Abstract base class for equity models."""

    def __init__(self, S0: float, r: float, q: float, T: float, n_paths: int, n_steps: int, rng: np.random.Generator) -> None:
        self.S0 = S0
        self.r = r
        self.q = q
        self.T = T
        self.n_paths = n_paths
        self.n_steps = n_steps
        self.dt = T / n_steps
        self.rng = rng

    @abstractmethod
    def simulate_paths(self) -> pl.DataFrame:
        """Simulate asset price paths and return as Polars DataFrame."""
        pass

# Monte Carlo Engine
Defines the engine

In [13]:
import numpy as np
import polars as pl

class MonteCarloEngine:
    """Monte Carlo engine for simulating Brownian paths."""

    def __init__(self, T: float, n_paths: int, n_steps: int, rng: np.random.Generator) -> None:
        self.T = T
        self.n_paths = n_paths
        self.n_steps = n_steps
        self.dt = T / n_steps
        self.sqrt_dt = np.sqrt(self.dt)
        self.rng = rng
        self.time_grid = np.linspace(0.0, T, n_steps + 1)

    def generate_brownian_increments(self) -> np.ndarray:
        """
        Generate standard Brownian motion increments dW for all paths and steps.
        Each path i has a sequence of random increments over time steps j:

            dW =
                   step_1   step_2   ...  step_N
            path_1   x11      x12    ...   x1N
            path_2   x21      x22    ...   x2N
            ...      ...      ...    ...   ...
            path_M   xM1      xM2    ...   xMN

        Each element x_ij represents one Brownian increment:
            dW_t = N(0,1) * sqrt(dt)

        Returns: array of shape (n_paths, n_steps)
        """
        dW = self.rng.standard_normal((self.n_paths, self.n_steps)) * self.sqrt_dt
        return dW

    def generate_brownian_paths(self) -> np.ndarray:
            """
            Generate Brownian motion paths W_t by cumulatively summing the increments dW.

            Each path i accumulates its increments over time steps j:

                dW =
                       step_1   step_2   ...  step_N
                path_1   x11      x12    ...   x1N
                path_2   x21      x22    ...   x2N
                ...      ...      ...    ...   ...
                path_M   xM1      xM2    ...   xMN

            The Brownian motion W_t is built as the cumulative sum of these increments.
            Including W_0 = 0

                W =
                        t0     t1       t2      ...    tN
                path_1   0    x11     x11+x12   ...   Sum x1j
                path_2   0    x21     x21+x22   ...   Sum x2j
                ...      ...    ...      ...    ...    ...
                path_M   0    xM1     xM1+xM2   ...   Sum xMj

            Mathematically:
                W_t = Sum_{k=1}^{t/dt} (N(0,1) * sqrt(dt))

            Returns: array of shape (n_paths, n_steps + 1)
            """
            dW = self.generate_brownian_increments()
            W = np.zeros((self.n_paths, self.n_steps + 1))
            W[:, 1:] = np.cumsum(dW, axis=1)
            return W



# Local Volatility Model

Theory:
- Step 1 - Define GBM Dynamic
- Step 2 - Define PDE with Local Volatility

What is found below is the **backward** pricing PDE (like Black-Scholes) but with a local volatility $\sigma_{\text{loc}}^2(S,t)$  that varies with both the underlying and time.

It's not the **forward** Dupire PDE!

SDE --> replication/martingale -->  backward pricing PDE --> (transformation TO DO) --> forward Dupire PDE.


## Step 1 - Risk-Neutral Dynamics and Pricing Expectation

Under the risk-neutral measure $\mathbb{Q}$, an equity paying a continuous dividend yield $q$ and earning the risk-free rate $r$ follows the stochastic differential equation (SDE):

$$
dS_t = S_t \big[(r - q)\,dt + \sigma_{\text{loc}}(S_t,t)\,dW_t\big].
$$

Here:  
- $S_t$ is the spot price at time $t$,  
- $\sigma_{\text{loc}}(S_t,t)$ is the local volatility, a deterministic function of spot and time,  
- $W_t$ is a standard Brownian motion under $\mathbb{Q}$.  

The drift term $(r - q)$ ensures that the discounted process $e^{-(r - q)t}S_t$ is a martingale, enforcing the no-arbitrage condition.  

For a European payoff $f(S_T)$ maturing at $T$, the time-$t$ price is given by the discounted expectation under $\mathbb{Q}$:  

$$
V(S_t,t) = e^{-r(T-t)}\,\mathbb{E}^{\mathbb{Q}}_t[f(S_T)].
$$

This expectation can be computed either with Monte Carlo simulation or by solving the associated PDE.  

| Model | Volatility | Dimension | Notes |  
|:--|:--|:--|:--|  
| Black–Scholes | Constant $\sigma$ | 1 D in $S$ | Simple but cannot reproduce volatility smiles |  
| Local Volatility | Deterministic $\sigma_{\text{loc}}(S,t)$ | 1 D in $S$ | Fits the entire implied-volatility surface for vanilla options |  


## Step 2 - From SDE to the Local-Volatility Pricing PDE (via Martingale)

0️⃣ *Set Up*  

Start from the risk-neutral SDE  
$$
dS_t = S_t\big[(r-q)\,dt + \sigma_{\text{loc}}(S_t,t)\,dW_t\big].
$$

Let $V(S,t)$ be the price of a European claim with maturity $T$ and payoff $f(S_T)$.  

Apply Itô’s lemma to $V(S_t,t)$:
$$
dV = V_t\,dt + V_S\,dS + \tfrac12 V_{SS}\,(dS)^2 .
$$

Because $(dS)^2 = S^2 \sigma_{\text{loc}}^2(S,t)\,dt$ and $dS$ has the form above, substitute:

$$
dV = \Big[V_t + (r-q)S\,V_S + \tfrac12 \sigma_{\text{loc}}^2(S,t) S^2\,V_{SS}\Big]dt
      + \sigma_{\text{loc}}(S,t) S V_S\, dW .
$$

1️⃣ *Discounted process*

Now consider the discounted process $ e^{-rt}V(S_t,t) $. 
1) If we want to enforce the no arbitrage condition, the proces $ e^{-rt}V(S_t,t) $ has to be have drift zero. 
2) To find the drift, we need to see how $ e^{-rt}V(S_t,t) $ over an infinitesimal step
3) So we compute its differential using the product rule and Itô’s lemma:  

$$
d(e^{-rt}V) = e^{-rt}\big(-rV\,dt + dV\big).
$$

Substitute the expression for $dV$:  

$$
d(e^{-rt}V) = e^{-rt}\big[
  (V_t + (r - q)S\,V_S + \tfrac{1}{2}\sigma_{\text{loc}}^2S^2V_{SS} - rV)\,dt
  + \sigma_{\text{loc}}(S,t)S\,V_S\,dW_t
\big].
$$

2️⃣  *No-arbitrage condition : drift zero*

Under the risk-neutral measure, discounted asset prices and all tradable portfolios must be martingales. 

For $e^{-rt}V(S_t,t)$ to be a martingale, its *drift term* (the part multiplying $dt$) must be zero.  

Setting that term to zero gives the local-volatility pricing PDE.

$$
\boxed{
V_t + (r - q)S\,V_S + \tfrac{1}{2}\sigma_{\text{loc}}^2(S,t)S^2V_{SS} - rV = 0.
}
$$

This PDE holds for any derivative price $V(S,t)$ under the local volatility model, with terminal condition $V(S,T) = f(S)$


The PDE says the time decay $V_t$ is balanced by drift convection $(r-q)S V_S$, curvature (gamma) weighted by local variance $\tfrac12\sigma_{\text{loc}}^2 S^2 V_{SS}$, and discounting $-rV$.

## Step 2 - From SDE to the Local-Volatility Pricing PDE (via Replication Argument)


0️⃣ *Set Up*  

Start from the risk-neutral SDE  
$$
dS_t = S_t\big[(r-q)\,dt + \sigma_{\text{loc}}(S_t,t)\,dW_t\big].
$$

Let $V(S,t)$ be the price of a European claim with maturity $T$ and payoff $f(S_T)$.  

Applying Itô’s lemma to $V(S_t,t)$ gives  
$$
dV = V_t\,dt + V_S\,dS + \tfrac12 V_{SS}\,(dS)^2 .
$$

Because $(dS)^2 = S^2 \sigma_{\text{loc}}^2(S,t)\,dt$ and $dS$ has the form above, substitute:  
$$
dV = \Big[V_t + (r-q)S\,V_S + \tfrac12 \sigma_{\text{loc}}^2(S,t) S^2\,V_{SS}\Big]dt
      + \sigma_{\text{loc}}(S,t) S V_S\, dW_t .
$$


1️⃣ *Build a hedged portfolio and differentiate*

Form a self-financing portfolio  
$$
\Pi_t = V(S_t,t) - \Delta_t S_t ,
$$  
where $\Delta_t$ is the number of underlyings held short to hedge the option.  
Differentiate: $d\Pi_t = dV - \Delta_t\, dS_t.$  

Substitute $dV$ and $dS_t$:
$$
\begin{aligned}
d\Pi_t
&= \Big[V_t + (r-q)S V_S + \tfrac12\sigma_{\text{loc}}^2S^2V_{SS}\Big]dt
   + \sigma_{\text{loc}} S V_S\, dW_t
   - \Delta_t S\big[(r-q)\,dt + \sigma_{\text{loc}}\,dW_t\big] \\
&= \Big[V_t + (r-q)S(V_S-\Delta_t) + \tfrac12\sigma_{\text{loc}}^2S^2V_{SS}\Big]dt
   + \sigma_{\text{loc}} S (V_S-\Delta_t)\, dW_t .
\end{aligned}
$$


2️⃣ Eliminate randomness (the hedge)  
Choose $\Delta_t = V_S$ so the stochastic term vanishes:
$$
d\Pi_t = \Big[V_t + \tfrac12\sigma_{\text{loc}}^2S^2V_{SS}\Big]dt .
$$
The portfolio is now locally risk-free (no $dW_t$).


3️⃣ *No-arbitrage condition : no risk only risk free*

A riskless portfolio must earn the risk-free rate $r$:
$$
d\Pi_t = r\,\Pi_t\,dt = r\,[V - S V_S]\,dt .
$$
Equating the two expressions for $d\Pi_t$:
$$
V_t + \tfrac12\sigma_{\text{loc}}^2S^2V_{SS} = r(V - S V_S) .
$$
Simplify:
$$
\boxed{
V_t + (r - q)S\,V_S + \tfrac12\sigma_{\text{loc}}^2(S,t)S^2V_{SS} - rV = 0 .
}
$$

The PDE states that for a hedged portfolio, all randomness is removed and the remaining deterministic evolution of value must earn the risk-free rate.

## Step 3 - Option Prices

Assumes call market prices are sourced from BBG or Reuters

In [14]:
# Call Market Prices

import polars as pl

# --- Spot and curve parameters ---
S0 = 100.0
r = 0.02
q = 0.01

# --- Grid data ---
strikes = [80, 90, 100, 110, 120]
maturities = [0.25, 0.5, 1.0, 2.0]

# --- Call price grid - Assumed to be market-observed ---
prices = [
    [20.23, 21.07, 22.78, 25.41],
    [11.08, 12.36, 14.43, 17.94],
    [3.81,  5.38,  7.97,  11.93],
    [0.92,  1.76,  3.41,  6.20],
    [0.18,  0.47,  1.20,  2.93],
]

# --- Build Polars DataFrame in long format ---
df = (
    pl.DataFrame(prices,
                 schema=[str(t) + "Y" for t in maturities],
                 orient="row")
    # Add a new column called "Strike"
    .with_columns(pl.Series("Strike", strikes))
    # Keep "Strike" fixed as the index (that’s the x-dimension),
    .unpivot(index=["Strike"], 
            # Take all the maturity columns (e.g. "0.25Y", "0.5Y", etc.),
            on=[str(t) + "Y" for t in maturities],
            # Stack them vertically into two new columns:
            # "Maturity" → will hold "0.25Y", "0.5Y", etc.
            # "CallPrice" → will hold the actual numerical prices.
            variable_name="Maturity", value_name="CallPrice")
    # Remove the letter “Y” (so "0.25Y" → "0.25")
    # Convert the column to float (0.25, 0.5, 1.0, 2.0)
    .with_columns(pl.col("Maturity").str.replace("Y", "").cast(pl.Float64))
)
df

Strike,Maturity,CallPrice
i64,f64,f64
80,0.25,20.23
90,0.25,11.08
100,0.25,3.81
110,0.25,0.92
120,0.25,0.18
…,…,…
80,2.0,25.41
90,2.0,17.94
100,2.0,11.93


## Step 4 - From Prices to Implied Volatilities

In [15]:
from scipy.optimize import brentq
from scipy.stats import norm
import numpy as np

# ------------------------------------------------------------
# 1. Black–Scholes call pricing function
# ------------------------------------------------------------
def bs_call_price(s0: float, K: float, r: float, q: float, sigma: float, T: float) -> float:
    if T <= 0.0:
        return max(0.0, s0 - K)
    vol_sqrt_t = sigma * np.sqrt(T)
    if vol_sqrt_t <= 0.0:
        return max(0.0, s0 * np.exp(-q * T) - K * np.exp(-r * T))
    d1 = (np.log(s0 / K) + (r - q + 0.5 * sigma**2) * T) / vol_sqrt_t
    d2 = d1 - vol_sqrt_t
    return s0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# ------------------------------------------------------------
# 2. Implied volatility solver (Brent Method)
# ------------------------------------------------------------
def implied_vol_call(price: float, s0: float, K: float, r: float, q: float, T: float,
                     sigma_lo: float = 1e-6, sigma_hi: float = 5.0) -> float:
    """Solve for implied volatility via Brent's method."""
    if T <= 0.0:
        return 0.0

    def f(sig: float) -> float:
        return bs_call_price(s0, K, r, q, sig, T) - price

    try:
        return brentq(f, sigma_lo, sigma_hi, maxiter=200)
    except ValueError:
        return np.nan

# ------------------------------------------------------------
# 3. Apply to your existing DataFrame `df`
# ------------------------------------------------------------
df_iv = df.with_columns(
    pl.struct(["Strike", "Maturity", "CallPrice"]).map_elements(
        lambda x: implied_vol_call(
            price=x["CallPrice"],
            s0=S0,
            K=x["Strike"],
            r=r,
            q=q,
            T=x["Maturity"]
        )
    ).alias("ImpliedVol")
)

df_iv


Strike,Maturity,CallPrice,ImpliedVol
i64,f64,f64,f64
80,0.25,20.23,0.223709
90,0.25,11.08,0.218528
100,0.25,3.81,0.185455
110,0.25,0.92,0.194339
120,0.25,0.18,0.205256
…,…,…,…
80,2.0,25.41,0.245137
90,2.0,17.94,0.216983
100,2.0,11.93,0.200333


In [16]:
import plotly.graph_objects as go


# Prepare data grid for plotting
strikes = df_iv["Strike"].unique().sort().to_numpy()
maturities = df_iv["Maturity"].unique().sort().to_numpy()

# Pivot to 2D matrix of implied volatilities
iv_matrix = (
    df_iv.to_pandas()
      .pivot(index="Strike", columns="Maturity", values="ImpliedVol")
      .sort_index(ascending=True)
      .to_numpy()
)

# Build meshgrid for the surface
T_grid, K_grid = np.meshgrid(maturities, strikes)

# --- Prepare red market data points ---
x_points = df_iv["Maturity"].to_numpy()
y_points = df_iv["Strike"].to_numpy()
z_points = df_iv["ImpliedVol"].to_numpy()

# Create interactive 3D surface + market dots
fig = go.Figure(data=[
    # Interpolated or gridded surface
    go.Surface(
        x=T_grid,
        y=K_grid,
        z=iv_matrix,
        colorscale="Viridis",
        opacity=0.8,
        contours={"z": {"show": True, "usecolormap": True, "highlightcolor": "limegreen"}},
        name="Interpolated Surface",
    ),
    # Actual market points
    go.Scatter3d(
        x=x_points,
        y=y_points,
        z=z_points,
        mode="markers",
        marker=dict(size=5, color="red", symbol="circle"),
        name="Market Data",
    )
])

# Layout and labels
fig.update_layout(
    title="Market Implied Volatilities Surface (with Observed Data Points)",
    scene=dict(
        xaxis_title="Maturity (Years)",
        yaxis_title="Strike",
        zaxis_title="Implied Volatilities",
    ),
    width=850,
    height=650,
    template="plotly_white",
)

fig.show()


## Step 5 - Interpolate Implied Volatility Surface§

Given option prices, local volatility surface could be computed by using derivatives of call prices. 
However before doing that there has to be some interpolation because current grid is coarse and derivatives on discrete data are not the best. So the next task is to choose the right interpolation method. Check the notebook [`Volatility Surface`](Volatility%20Surface%20|%20Interpolation%20Methods.ipynb) for more info about the interpolation.

For Dupire Surface we are going to use a Bivariate Cubic Spline

In [17]:
import numpy as np
from scipy import interpolate
import plotly.graph_objects as go

# --------------------------------------------------------------------------
# --- Convert inputs to numpy arrays
strikes = np.array(strikes)
maturities = np.array(maturities)

# --------------------------------------------------------------------------
# --- Create a fine meshgrid for interpolation
# 1) Fine grid of evaluation points --> 100×100 grid for surface
# These are going to be used in `bisplev` / `RectBivariateSpline`
K_fine = np.linspace(strikes.min(), strikes.max(), 100)
T_fine = np.linspace(maturities.min(), maturities.max(), 100)

# 2) Build 2D matrices of coordinates
# These are going to be used for the 3D surface plot
K_grid, T_grid = np.meshgrid(K_fine, T_fine)

# --------------------------------------------------------------------------
# --- Flatten for `bisplrep` (requires 1D x, y, z arrays)
# Used also for plotting the red market data dots
T_flat, K_flat = np.meshgrid(maturities, strikes)   # maturity first, strike second
x = T_flat.ravel()   # maturities (x-axis)
y = K_flat.ravel()   # strikes (y-axis)
z = iv_matrix.ravel()   # Implied Volatilities (z-axis)

# --------------------------------------------------------------------------
# --- Interpolation (RectBivariateSpline)
# RectBivariateSpline builds a smooth 2D spline on a rectangular grid.
# Note: we transpose `prices` to match (maturities, strikes) orientation.
bivar_cubic = interpolate.RectBivariateSpline(maturities, strikes, iv_matrix.T)

# Evaluate the fitted surface on the fine grid (T_fine, K_fine) created earlier
C_bivar = bivar_cubic(T_fine, K_fine)

# --------------------------------------------------------------------------
# --- Create single 3D figure
fig = go.Figure()

# Add interpolated surface
fig.add_trace(
    go.Surface(
        x=T_grid,               # maturities
        y=K_grid,               # strikes
        z=C_bivar,              # interpolated call prices
        colorscale="Viridis",
        opacity=0.85,
        name="Interpolated Surface"
    )
)

# Add market data points (red dots)
fig.add_trace(
    go.Scatter3d(
        x=x, y=y, z=z,
        mode="markers",
        marker=dict(size=4, color="red"),
        name="Market data"
    )
)

# --------------------------------------------------------------------------
# --- Layout configuration
# Control figure size, margins, camera, and 3D scene axes
fig.update_layout(
    title_text="RectBivariateSpline Interpolated Surface",
    width=1000,   # wider figure
    height=500,  # taller figure
    template="plotly_white",
    margin=dict(l=40, r=20, b=60, t=40),  # prevent clipping of labels
    scene=dict(
        xaxis_title="Maturity (Years)",
        yaxis_title="Strike",
        zaxis_title="Call Price",
        camera=dict(
            eye=dict(x=1.6, y=1.8, z=0.9)  # set viewing angle
        )
    )
)

# --------------------------------------------------------------------------
# --- Show figure
fig.show()


## Step 6 - Log Monenynes and Total Variance - Theory


### 📕 Use log-moneyness $k$ instead of strike $K$

We define  

$$
k = \ln\left(\frac{K}{F_T}\right), \quad \text{with} \quad F_T = S_0 e^{(r - q)T}.
$$

This normalizes the strike by the forward price so the horizontal axis represents **how far from at-the-money (ATM)** the option is, in a **dimensionless** way.  

- Volatility smiles are much smoother and more symmetric when expressed in $k$ rather than in raw $K$.  
- Interpolating in $k$ avoids distortions caused by the exponential relation between $K$ and prices and makes cross-maturity comparisons consistent — **ATM is always at $k=0$**.
- Think of $k=0$ as ATM, negative k as ITM call / OTM put, positive k as OTM call.

#### What "dimensionless" means

When we say **"dimensionless"**, we mean the variable has **no physical units** (like dollars, years, etc.).

Here’s why that matters:

- Strike $K$ has **units of price** (e.g. \$100).  
- The forward $F_T$ also has **units of price** (e.g. \$105).  
- When we divide $K / F_T$, the units cancel — it’s a *pure ratio*.  
- Taking the log of that ratio,  
  $$
  k = \ln\left(\frac{K}{F_T}\right),
  $$
  still has no units — it’s **dimensionless**.

This is powerful because it lets us compare options **across different assets or maturities**, even if their price scales are completely different. For example, $k = 0.1$ (≈ 10% out-of-the-money call) means the same relative moneyness whether the stock price is \$10 or \$1000.


### 📕 Use total variance instead of raw implied volatility

We define the **total variance** as  

$$
w(k, T) = \sigma_{\text{BS}}(k, T)^2 \, T
$$

where $\sigma_{\text{BS}}(k, T)$ is the Black–Scholes implied volatility for log-moneyness $k$ and maturity $T$.

#### Why we use total variance

- **Smoother across time:**  
  The function $w(k, T)$ tends to vary *linearly* with $T$ for many markets, while $\sigma_{\text{BS}}(k, T)$ does not. That makes interpolation and extrapolation across maturities more stable.

- **Mathematically natural:**  
  In Dupire’s equation for local volatility, the time derivative appears as $\partial w / \partial T$. This means $w(k, T)$ is the quantity that enters directly into the PDE, not $\sigma$.

- **Always positive:**  
  Since both $\sigma^2$ and $T$ are positive, $w(k, T)$ naturally satisfies $w \ge 0$.

- **Consistent scaling:**  
  Expressing volatility in variance-time units keeps the same magnitude whether $T$ is small or large. Short-maturity options no longer have artificially large noise in $\sigma$ due to division by $\sqrt{T}$ in $d_1$. Basically, it could happened that volatility is impacted only by short maturity more than by the variance itself.

In practice, we will interpolate and smooth **$w(k, T)$** instead of $\sigma_{\text{BS}}(k, T)$,  
and later recover the implied volatility via  

$$
\sigma_{\text{BS}}(k, T) = \sqrt{\frac{w(k, T)}{T}}.
$$


In [18]:
import math
import numpy as np
import polars as pl

# Define Forward Price (F)
def forward_price(s0: float, r: float, q: float, T: float) -> float:
    return s0 * math.exp((r - q) * T)

# Logm Moneynes -->  normalise strike by forward
def log_moneyness(K: float, F: float) -> float:
    return math.log(K / F)

# Total Variance --> smoother quantity in time
def total_variance(iv: float, T: float) -> float:
    return (iv * iv) * max(T, 1e-12)

# Implied Volatility from total variance
def iv_from_total_variance(w: float, T: float) -> float:
    return math.sqrt(max(w, 0.0) / max(T, 1e-12)) # Avoiding division by zero

## Step 7 - Log Monenynes and Total Variance - Practice
For each maturity $T_i$:

1. **Compute the forward price**
   $$
   F_T = S_0 e^{(r - q)T_i}
   $$

2. **Transform strikes into log-moneyness**
   $$
   k_j = \ln\left(\frac{K_j}{F_T}\right)
   $$

3. **Compute total variance**
   $$
   w_j = \sigma_{\text{BS}}(K_j, T_i)^2 \, T_i
   $$

4. **Store** the pairs $(k_j, w_j)$ for each maturity $T_i$.

In [ ]:
import polars as pl

# Implied Volatilities Interpolated --> df_iv
df_kw_list: list[pl.DataFrame] = []

# To check [(T_val,df_slice) for T_val, df_slice in df_iv.group_by("Maturity", maintain_order=True)]
for T_value, df_iv_slice in df_iv.group_by("Maturity", maintain_order=True):

    # Compute forward for each maturity 
    # T_value is a tuple so [0]
    F_T = forward_price(S0, r, q, T_value[0])
    
    # Apply log-moneyness and total variance transformations
    df_kw = (
        df_iv_slice
        .with_columns([
            # pl.lit creates a constant column filled with F_T and broadcast it across all rows
            pl.lit(F_T).alias("Forward"),
            (pl.col("Strike") / pl.lit(F_T)).log().alias("k"),               # log-moneyness
            (pl.col("ImpliedVol") ** 2 * pl.lit(T_value[0])).alias("w"),     # total variance
        ])
        .select(["Maturity", "Strike", "Forward", "k", "w"])
        .sort("k")
    )
    df_kw_list.append(df_kw)

# Combine all slices
df_kw_all = pl.concat(df_kw_list, how="vertical")

df_kw_all

Maturity,Strike,Forward,k,w
f64,i64,f64,f64,f64
0.25,80,100.250313,-0.225644,0.012511
0.25,90,100.250313,-0.107861,0.011939
0.25,100,100.250313,-0.0025,0.008598
0.25,110,100.250313,0.09281,0.009442
0.25,120,100.250313,0.179822,0.010533
…,…,…,…,…
2.0,80,102.020134,-0.243144,0.120184
2.0,90,102.020134,-0.125361,0.094164
2.0,100,102.020134,-0.02,0.080266


## Step 8- Calendar-Arbitrage Check

We verify that total variance $w(k, T) = \sigma_{\text{BS}}(k, T)^2 \, T$ is **monotonic in maturity** for each log-moneyness $k$.

- The condition for no calendar arbitrage is:
  $$
  \frac{\partial w(k, T)}{\partial T} \ge 0
  $$
  for all $k$ and $T$.

- Intuitively, this means that as maturity increases, the option’s total variance (and therefore its time value) should not decrease.

- In the plot **Total Variance vs Log-Moneyness by Maturity**, the curves are smoothly ordered by maturity:
  - Shorter maturities (e.g. 0.25y) lie below longer maturities (e.g. 2.00y).  
  - No crossing of curves is visible, confirming monotonicity.

The total variance surface is free from calendar arbitrage


### Why we check calendar arbitrage on total variance instead of directly on call prices

In theory, calendar arbitrage means that for every strike $K$, the call price should not decrease as maturity increases:

$$
C_T(K, T) = \frac{\partial C(K, T)}{\partial T} \ge 0.
$$

However, in the Black–Scholes framework, the call price depends on maturity mainly through the **total variance**
$$
w(k, T) = \sigma_{\text{BS}}(k, T)^2 \, T,
$$
where $k = \ln(K / F_T)$ is log-moneyness.

Using the chain rule,
$$
\frac{\partial C}{\partial T}
= \frac{\partial C}{\partial w} \cdot \frac{\partial w}{\partial T}.
$$

The first term $\frac{\partial C}{\partial w}$ is always positive:
$$
\frac{\partial C}{\partial w}
= \text{Vega} \cdot \frac{1}{2\sigma T} > 0.
$$

Therefore, the **sign of $C_T$** is fully determined by $\frac{\partial w}{\partial T}$:
$$
C_T(K, T) \ge 0
\quad\Longleftrightarrow\quad
\frac{\partial w(k, T)}{\partial T} \ge 0
\quad (\text{with } k \text{ fixed}).
$$

This means that ensuring monotonicity of total variance $w(k, T)$ across maturities
is **equivalent** to ensuring that option prices are non-decreasing in maturity.



In [31]:
import plotly.graph_objects as go

# --- Create figure ---
fig = go.Figure()

# One line per maturity
for T_val, df_slice in df_kw_all.group_by("Maturity", maintain_order=True):
    T = float(T_val[0])
    dfp = df_slice.sort("k")
    fig.add_trace(
        go.Scatter(
            x=dfp["k"],
            y=dfp["w"],
            mode="lines+markers",
            name=f"T={T:.2f}y"
        )
    )

# --- Layout ---
fig.update_layout(
    title="Total Variance vs Log-Moneyness by Maturity",
    xaxis_title="log-moneyness  k = ln(K / F_T)",
    yaxis_title="total variance  w = sigma^2T",
    template="plotly_dark",
    width=1000,
    height=600
)

fig.show()


TO DO:

- Check for butterfly arbitrage
- Once we have these $(k, w)$ slices for every expiry, we will apply interpolation in $k$ for each fixed $T_i$ to obtain a smooth and stable function $w(k, T_i)$.
